# **Random Grid Scenario Preparation**

From the output of `activitygen` this notebook generates a parking lot augmented scenario in the following way:

- `randUni` traffic will be repeated each day (random departure can be provided by SUMO)
- `hXcY` household traffic: instead of individual trips the car will complete exactly one trip with many stops at the defined edges (during night a random depart is added)
- `carInx` commuter traffic: the original trip will be repeated for each day, but the individual trips within a day will be replaced by a single trip with a single stop (random departure can be provided by SUMO)

In [1]:
import pandas as pd
from tqdm.notebook import trange

import numpy as np

In [2]:
DAYS = 10 #number of days to generate
ROOT = "../../02_simulation/random_grid/"

In [3]:
#for analyzing the movements:
trips = pd.read_xml(ROOT+"gen_activities.trips.xml", xpath="trip")
trips = trips[~trips["id"].str.startswith("randUni")]
trips["veh_id"] = trips["id"].transform(lambda x: x.split(":")[0])
trips["move_id"] = trips["id"].transform(lambda x: int(x.split(":")[1]))

In [4]:
trips.head(5)

,id,type,depart,departPos,arrivalPos,arrivalSpeed,from,to,via,veh_id,move_id
129,h3437c1:1,default,21594.0,60.60,60.60,0.0,202,202,196,h3437c1,1
130,h3359c1:1,default,21595.0,124.11,124.11,0.0,68,68,172,h3359c1,1
131,h3449c1:1,default,21595.0,89.84,89.84,0.0,202,202,-172,h3449c1,1
132,carIn186:1,default,21595.0,5.00,78.30,0.0,-338,-152,None,carIn186,1
133,carIn194:1,default,21595.0,5.00,35.06,0.0,-338,-162,None,carIn194,1


In [5]:
max_trips = trips.groupby(by="veh_id")["move_id"].max()
first_depart = trips.groupby(by="veh_id")["depart"].min()

In [6]:
def get_next_depart_time(trip_df, veh_id, move_id):
    moves = trip_df[trip_df["veh_id"] == veh_id]
    #as trip_df is ordered by moving times, we search for the first move_id which is greater than actual:
    for i,r in moves.iterrows():
        if r["move_id"] > move_id:
            return r["depart"]
    return -1

def get_stop_duration(trip_df, veh_id, move_id):
    moves = trip_df[trip_df["veh_id"] == veh_id]
    #as trip_df is ordered by moving times, we search for the first move_id which is greater than actual:
    start = 0
    for i,r in moves.iterrows():
        if r["move_id"] > move_id:
            return r["depart"]-start
        elif r["move_id"] == move_id:
            start = r["depart"]
    return -1

In [7]:
def generate_stops_households(trip_df, veh_id, days=1):
    '''
        For households, this function replaces trips by stops at the given destinations.
    '''
    moves = trip_df[trip_df["veh_id"] == veh_id]
    home = moves.iloc[0]
    
    new_trip = ET.Element("trip")
    new_trip.set("id", "%s:0"%(veh_id))
    new_trip.set("type", "default")
    new_trip.set("depart", str(home["depart"]))
    new_trip.set("departPos", str(home["departPos"]))
    new_trip.set("arrivalPos", str(home["arrivalPos"]))
    new_trip.set("from", str(home["from"]))
    new_trip.set("to", str(home["from"]))
    
    for day in range(days):
        for i,r in moves.iterrows():
            new_stop = ET.SubElement(new_trip, "stop")
            new_stop.set("parkingArea", "pa%s"%r["to"])
            
            move_id = int(r["move_id"])
            stop_duration = get_stop_duration(moves, veh_id, move_id)
            if stop_duration == -1: #last trip
                stop_duration = np.random.normal(home["depart"]+24*60*60 - r["depart"], 7*60, 1)[0] #staying at home for a random time
                
            new_stop.set("duration", str(stop_duration))
            
    return new_trip

def generate_stops_commuters(trip_df, veh_id, day):
    moves = trip_df[trip_df["veh_id"] == veh_id]
    home = moves.iloc[0]
    out = moves.iloc[-1]
    
    departure_time = float(home["depart"]) + 24*60*60*day
    new_trip = ET.Element("trip")
    new_trip.set("id", "%s:%d"%(veh_id, day))
    new_trip.set("type", "default")
    new_trip.set("depart", str(departure_time))
    new_trip.set("departPos", str(home["departPos"]))
    new_trip.set("arrivalPos", str(home["arrivalPos"]))
    new_trip.set("from", str(home["from"]))
    new_trip.set("to", str(out["to"]))
    
    #adding parking during the day:
    move_id = home["move_id"]
    parking_time = get_stop_duration(moves, veh_id, move_id)
    new_stop = ET.SubElement(new_trip, "stop")
    new_stop.set("duration", str(parking_time))
    new_stop.set("parkingArea", "pa%s"%home["to"])
    
    return new_trip
    

In [8]:
import xml.etree.ElementTree as ET
from xml.dom import minidom

def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ET.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

In [10]:
trips_tree = ET.parse(ROOT+"gen_activities.trips.xml")
trips_with_parking_tree = ET.Element("routes")
processed_households = set() #one household car shall be processed only once _at all_

for day in trange(DAYS):
    processed_commuters = set() #one commuter car shall be processed only once _per day_
    for elem in trips_tree.getroot():
        if elem.tag != "trip":
            if day == 0: trips_with_parking_tree.insert(-1, elem)
        else:
            trip_id = elem.get("id")
            veh_id, trip_no = trip_id.split(":")

            if veh_id.startswith("randUni"):
                #simply adding uniform random traffic: (for each day)
                trip_type = elem.get("type")
                trip_depart_new = float(elem.get("depart"))+day*24*60*60 #departure on the next day
                trip_depart_pos = elem.get("departPos")
                trip_arrival_pos = elem.get("arrivalPos")
                trip_arrival_speed = elem.get("arrivalSpeed")
                trip_from = elem.get("from")
                trip_to = elem.get("to")

                duplicate_move = ET.SubElement(trips_with_parking_tree, "trip")
                duplicate_move.set("id", "%s:%d"%(veh_id, day))
                duplicate_move.set("type", trip_type)
                duplicate_move.set("depart", str(trip_depart_new))
                duplicate_move.set("departPos", trip_depart_pos)
                duplicate_move.set("arrivalPos", trip_arrival_pos)
                duplicate_move.set("arrivalSpeed", trip_arrival_speed)
                duplicate_move.set("from", trip_from)
                duplicate_move.set("to", trip_to)
            
            if (veh_id.startswith("h")) and (day == 0) and (not(veh_id in processed_households)):
                #households:
                trip_with_stops = generate_stops_households(trips, veh_id, days=DAYS)
                trips_with_parking_tree.insert(len(trips_with_parking_tree), trip_with_stops)
                processed_households.add(veh_id)
                
            if (veh_id.startswith("carIn")) and (not(veh_id in processed_commuters)):
                #commuters:
                commuter_stops = generate_stops_commuters(trips, veh_id, day)
                trips_with_parking_tree.insert(len(trips_with_parking_tree), commuter_stops)
                processed_commuters.add(veh_id)

  0%|          | 0/10 [00:00<?, ?it/s]

In [35]:
#saving:

with open(ROOT+"burnin_trips.trip.xml", "w") as f:
    f.write(prettify(trips_with_parking_tree))